In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from LSTM import *
import pandas as pd
import numpy as np
import torch.utils.data as utils
import time
import os
import pickle

import pdb

# For the notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

torch.manual_seed(1)

## TRAIN SET

In [2]:
gt_train_coord = pickle.load( open( "./import_dataset_3/train/gt_train_coord.pkl", "rb" ) )
gt_train  = pickle.load( open( "./import_dataset_3/train/gt_train.pkl", "rb" ) )
in_train_coord  = pickle.load( open( "./import_dataset_3/train/in_train_coord.pkl", "rb" ) )
inputs_train = pickle.load( open( "./import_dataset_3/train/inputs_train.pkl", "rb" ) )

inputs_train = torch.from_numpy(inputs_train).float()
gt_train = torch.from_numpy(gt_train).float()


## VALIDATION SET

In [3]:
gt_validation_coord = pickle.load( open( "../import_dataset_3/validation/gt_validation_coord.pkl", "rb" ) )
gt_validation = pickle.load( open( "../import_dataset_3/validation/gt_validation.pkl", "rb" ) )
in_validation_coord = pickle.load( open( "../import_dataset_3/validation/in_validation_coord.pkl", "rb" ) )
inputs_validation = pickle.load( open( "../import_dataset_3/validation/inputs_validation.pkl", "rb" ) )

inputs_validation = torch.from_numpy(inputs_validation).float()
gt_validation = torch.from_numpy(gt_validation).float()

### Model

In [4]:
from torch import optim

lr = 0.01
lstm = LSTM(input_size = inputs_train.shape[2], output_size = 4, num_layers=2, hidden_size=128)
optimizer = optim.Adam(lstm.parameters(), lr=lr,weight_decay=1e-4)
criterion = nn.MSELoss()

In [ ]:
traindataset = utils.TensorDataset(inputs_train, gt_train[:,1:,:])
trainloader = utils.DataLoader(traindataset, batch_size=16, shuffle=True)

valdataset = utils.TensorDataset(inputs_validation, gt_validation[:,1:,:])
valloader = utils.DataLoader(valdataset, batch_size=16, shuffle=True)

epochs = 150
steps = 0
print_every = 323
running_loss = 0 

loss_train = []
loss_val = []
for e in range(epochs):
    start = time.time()
    total_train_loss=0
    steps_bis = 0
    if (e+1)%10==0:
        lr /= 2
        optimizer = optim.Adam(lstm.parameters(), lr=lr, weight_decay=1e-4)

    for train_coord, ground_tru in iter(trainloader):
        steps += 1
        steps_bis+=1
        
        train_coord = train_coord.permute([1,0,2])
        ground_tru = ground_tru.permute([1,0,2])

        in_train = Variable(train_coord)
        targets = Variable(ground_tru)
        optimizer.zero_grad()
        #print(in_train.shape)
        #print(targets.shape)
        out = lstm.forward(in_train)
    
        #pdb.set_trace()
        loss1 = (criterion(out[:,:,0:2], targets[:,:,0:2]))
        loss2 = (criterion(out[:,:,2:], targets[:,:,2:]))

        loss1.backward(retain_graph=True)
        loss2.backward()
        optimizer.step()
        
        running_loss += (loss1+loss2).item()
        total_train_loss += (loss1+loss2).item()
    
        if steps % print_every == 0:
                stop = time.time()
                val_loss=0
                for ii, (valcoord, valgt) in enumerate(valloader):
                    valcoord = valcoord.permute([1,0,2])
                    valgt = valgt.permute([1,0,2])
                    inputs = Variable(valcoord, volatile=True)
                    predicted = lstm.predict(inputs)
                    #
                    val_loss+= (criterion(predicted[:,:,0:2],valgt[:,:,0:2]).item()
                                + criterion(predicted[:,:,2:],valgt[:,:,2:]).item())
                    
                print("Epoch: {}/{}..".format(e+1, epochs),
                  "Validation loss: {:.4f}..".format(val_loss/ii),
                  "Training loss: {:.4f}..".format(running_loss/print_every),
                  "{:.4f} s/batch".format((stop - start)/print_every)
                 )
                loss_val.append(val_loss/ii)
                running_loss = 0
                start = time.time()
    loss_train.append(total_train_loss/steps_bis)

/Users/rodolphefarrando/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Epoch: 1/150.. Validation loss: 1.1996.. Training loss: 2.0654.. 0.0549 s/batch
Epoch: 2/150.. Validation loss: 1.1857.. Training loss: 1.0916.. 0.0469 s/batch
Epoch: 3/150.. Validation loss: 0.9610.. Training loss: 0.9755.. 0.0419 s/batch
Epoch: 4/150.. Validation loss: 1.0519.. Training loss: 0.8991.. 0.0427 s/batch
Epoch: 5/150.. Validation loss: 1.0013.. Training loss: 0.8823.. 0.0448 s/batch
Epoch: 6/150.. Validation loss: 0.9907.. Training loss: 0.8330.. 0.0445 s/batch
Epoch: 7/150.. Validation loss: 0.9008.. Training loss: 0.8172.. 0.0433 s/batch
Epoch: 8/150.. Validation loss: 0.8874.. Training loss: 0.7972.. 0.0429 s/batch
Epoch: 9/150.. Validation loss: 0.8927.. Training loss: 0.7711.. 0.0425 s/batch
Epoch: 10/150.. Validation loss: 0.8385.. Training loss: 0.6815.. 0.0421 s/batch
Epoch: 11/150.. Validation loss: 0.8010.. Training loss: 0.6406.. 0.0409 s/batch
Epoch: 12/150.. Validation loss: 0.8189.. Training loss: 0.6264.. 0.0422 s/batch
Epoch: 13/150.. Validation loss: 0.81

In [ ]:
epoch = np.arange(1,epochs+1)
plt.figure(figsize=(12, 7))
plt.rc('font', family='serif')
plt.rc('font', size=20)

plt.plot(epoch,loss_train,label='Training loss')
plt.plot(epoch,loss_val,c='k',label='Validation loss')
plt.xlabel('epoch')
plt.ylabel('MSE error')
plt.legend()
plt.savefig(r'./figures/loss_2Loss_ND.pdf', bbox_inches='tight')

In [ ]:
torch.save(lstm.state_dict(), '2Loss_ND_3.pt')